In [3]:
import pandas as pd

# Charger le dataset
file_path = "data/spamdata_v2.csv"
df = pd.read_csv(file_path)

# Afficher les premières lignes et les infos générales
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   int64 
 1   text    5572 non-null   object
dtypes: int64(1), object(1)
memory usage: 87.2+ KB


(None,
    label                                               text
 0      0  Go until jurong point, crazy.. Available only ...
 1      0                      Ok lar... Joking wif u oni...
 2      1  Free entry in 2 a wkly comp to win FA Cup fina...
 3      0  U dun say so early hor... U c already then say...
 4      0  Nah I don't think he goes to usf, he lives aro...)

In [1]:
!pip install transformers

In [4]:
from transformers import BertTokenizer

# Charger le tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenizer un exemple pour voir le résultat
example_text = df["text"][0]
tokens = tokenizer(example_text, padding="max_length", truncation=True, max_length=128)

tokens

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': [101, 2175, 2127, 18414, 17583, 2391, 1010, 4689, 1012, 1012, 2800, 2069, 1999, 11829, 2483, 1050, 2307, 2088, 2474, 1041, 28305, 1012, 1012, 1012, 25022, 2638, 2045, 2288, 26297, 28194, 1012, 1012, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [5]:
import torch
from torch.utils.data import Dataset

# Recharger le tokenizer maintenant que transformers est installé
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Définition d'un dataset PyTorch pour BERT
class SpamDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenisation
        encoding = self.tokenizer(text,
                                  padding="max_length",
                                  truncation=True,
                                  max_length=self.max_len,
                                  return_tensors="pt")

        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Création du dataset
dataset = SpamDataset(df["text"].tolist(), df["label"].tolist(), tokenizer)

# Exemple d'élément tokenisé
dataset[0]

{'input_ids': tensor([  101,  2175,  2127, 18414, 17583,  2391,  1010,  4689,  1012,  1012,
          2800,  2069,  1999, 11829,  2483,  1050,  2307,  2088,  2474,  1041,
         28305,  1012,  1012,  1012, 25022,  2638,  2045,  2288, 26297, 28194,
          1012,  1012,  1012,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [6]:
from torch.utils.data import DataLoader, random_split

# Définir la taille des jeux de données (80% train, 20% validation)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Séparer en train et validation
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Définir les DataLoaders
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Vérifier un batch
next(iter(train_loader))

{'input_ids': tensor([[ 101, 4931, 1045,  ...,    0,    0,    0],
         [ 101, 1055, 1024,  ...,    0,    0,    0],
         [ 101, 7592, 1010,  ...,    0,    0,    0],
         ...,
         [ 101, 2204, 5027,  ...,    0,    0,    0],
         [ 101, 7929, 8840,  ...,    0,    0,    0],
         [ 101, 1045, 1005,  ...,    0,    0,    0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'label': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [8]:
from transformers import BertForSequenceClassification

# Charger le modèle BERT avec une couche de classification binaire
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Déplacer le modèle sur GPU si disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Vérifier que le modèle est bien sur le GPU
print(f"Modèle chargé sur : {device}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Modèle chargé sur : cuda


In [10]:
from torch.optim import AdamW  # Import depuis torch

# Définir l'optimiseur (AdamW recommandé pour BERT)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Définir la fonction de perte (cross-entropy pour classification binaire)
loss_fn = torch.nn.CrossEntropyLoss()

# Nombre d'époques d'entraînement
num_epochs = 3


In [11]:
from tqdm import tqdm

# Fonction d'entraînement
def train_model(model, train_loader, val_loader, optimizer, loss_fn, num_epochs, device):
    model.train()  # Mode entraînement

    for epoch in range(num_epochs):
        total_loss = 0
        correct, total = 0, 0

        loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

        for batch in loop:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            optimizer.zero_grad()

            # Passage dans le modèle
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs.logits, labels)

            # Rétropropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Calcul de la précision
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            loop.set_postfix(loss=loss.item(), accuracy=correct/total)

        print(f"\nEpoch {epoch+1}: Loss = {total_loss/len(train_loader):.4f}, Accuracy = {correct/total:.4f}")

        # Évaluation sur validation
        evaluate_model(model, val_loader, device)

# Fonction d'évaluation
def evaluate_model(model, val_loader, device):
    model.eval()  # Mode évaluation
    correct, total = 0, 0

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy:.4f}")

# Lancer l'entraînement
train_model(model, train_loader, val_loader, optimizer, loss_fn, num_epochs, device)


Epoch 1/3: 100%|██████████| 279/279 [01:38<00:00,  2.83it/s, accuracy=0.975, loss=0.0204]



Epoch 1: Loss = 0.0842, Accuracy = 0.9746
Validation Accuracy: 0.9946


Epoch 2/3: 100%|██████████| 279/279 [01:36<00:00,  2.89it/s, accuracy=0.995, loss=0.0128]



Epoch 2: Loss = 0.0214, Accuracy = 0.9946
Validation Accuracy: 0.9812


Epoch 3/3: 100%|██████████| 279/279 [01:36<00:00,  2.89it/s, accuracy=0.998, loss=0.00047]



Epoch 3: Loss = 0.0070, Accuracy = 0.9982
Validation Accuracy: 0.9946
